In [1]:
import glob
import os
import numpy as np
import pandas as pd
import zipfile

In [2]:
path = "C:\\Users\\linathan\\Documents\\cleaned_files_dec2020_nov2025"
traincsv = []

In [4]:
june2026 = pd.read_csv("C:\\Users\\linathan\\Documents\\cleaned_files_dec2020_nov2025\\cleaned_2021_6.csv")

In [5]:
june2026 = june2026.drop('Unnamed: 0', axis=1)
june2026 = june2026[june2026['Cancelled'] ==0]
june2026.describe()
print(june2026[june2026.isnull().any(axis=1)], "\n")

        Year  Month  DayofMonth  DayOfWeek  CRSDepTime  DepTime  DepDelay  \
362944  2021      6          21          1        2059   2137.0      38.0   

        DepDelayMinutes  DepDel15  CRSArrTime  ...  AirTime  WeatherDelay  \
362944             38.0       1.0        2304  ...      NaN           0.0   

        NotWeatherDelay  Dest  Origin  Reporting_Airline  Tail_Number  TaxiIn  \
362944              0.0   CLE     BOS                 YX       N871RW     NaN   

       TaxiOut CancelOrDelayed  
362944    64.0            True  

[1 rows x 25 columns] 



In [6]:
june2026= june2026.dropna()
for col in june2026.select_dtypes(include = ['int64', 'float64']).columns:
            june2026[col] = june2026[col].astype(np.int16)

In [3]:
#fills the list traincsv with the dataframes of dec 2020 to nov 2024, which will be the training set
#due to how the cleanedcsv files were saved/cleaned, the first column of every dataframes is extraneous and will be removed
#for now, i am removing any cancelled flights
#numbers are being cast to 
for year in range(2020,2025):
    minMonth = 12 if year == 2020 else 1
    maxMonth = 11 if year == 2024 else 12
    for month in range(minMonth, maxMonth+1):
        df = pd.read_csv(path + f'\\cleaned_{year}_{month}' +'.csv')

        #the following lines will remove the first column of df, cancelled flights, and downcast ints to int16 to save memory
        #some noncancelled flights are mysteriously missing data, so they are dropped
        df = df.drop('Unnamed: 0', axis=1)
        df = df[df['Cancelled'] ==0]
        df = df.dropna()
        for col in df.select_dtypes(include = ['int64', 'float64']).columns:
            df[col] = df[col].astype(np.int16)
        traincsv.append(df)
        print(f'{year}_{month}')

2020_12
2021_1
2021_2
2021_3
2021_4
2021_5


IntCastingNaNError: Cannot convert non-finite values (NA or inf) to integer

In [14]:
traincsv[0].dtypes

Unnamed: 0             int64
Year                   int64
Month                  int64
DayofMonth             int64
DayOfWeek              int64
CRSDepTime             int64
DepTime              float64
DepDelay             float64
DepDelayMinutes      float64
DepDel15             float64
CRSArrTime             int64
ArrTime              float64
ArrDelay             float64
ArrDelayMinutes      float64
ArrDel15             float64
Cancelled            float64
AirTime              float64
WeatherDelay         float64
NotWeatherDelay      float64
Dest                  object
Origin                object
Reporting_Airline     object
Tail_Number           object
TaxiIn               float64
TaxiOut              float64
CancelOrDelayed         bool
dtype: object

In [13]:
#concatenates the list of csv's into one df
traindf = pd.concat(traincsv)

MemoryError: Unable to allocate 1.49 GiB for an array with shape (8, 24980535) and data type float64

In [6]:
## Extract input and output data as dataframe
X = li[0][['Reporting_Airline','Origin','Dest', 'ArrDelayMinutes']]
X = X.dropna(how = 'any')

In [7]:
## Split data into train and test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X[['Reporting_Airline','Origin','Dest']], X[['ArrDelayMinutes']], 
                                                    shuffle=True, random_state=627, test_size=0.2)

In [8]:
## Create an instance of the dummy regressor, which always predicts the mean of the training data. i.e., the mean of delayed time
## The Dummy classifier is not meaningful, because most flighs are not cancelled or delayed.
## Thus, the dummy classifier will always predict that the flight is on time.
from sklearn.dummy import DummyRegressor
dummy_regr = DummyRegressor(strategy="mean")

In [9]:
#score records the cross validation mean accuracy
score = np.zeros((5))

In [21]:
##k-fold cross validation
from sklearn.model_selection import KFold
kfold = KFold(n_splits=5,shuffle=True, random_state=626)

for i, (train_index, test_index) in enumerate(kfold.split(X_train, y_train)):
    ## get the kfold training data
    X_train_train = X_train.iloc[train_index, :]
    y_train_train = y_train.iloc[train_index]

    ## get the holdout data
    X_holdout = X_train.iloc[test_index, :]
    y_holdout = y_train.iloc[test_index]

    ## Fit using Dummy Regressor
    dummy_regr.fit(X_train_train, y_train_train)
    
    ## Generate predictions on the holdout set
    dummy_pred = dummy_regr.predict(X_holdout)    

    ## Record mean accuracy of self.predict(X) w.r.t. y as a float
    score[i] = dummy_regr.score(X_holdout, y_holdout) 

In [22]:
score

array([-2.53914004e-06, -6.50027038e-07, -1.50723500e-09, -1.04439616e-05,
       -3.32961073e-05])

In [23]:
X = li[0][['Reporting_Airline','Origin','Dest', 'ArrDelayMinutes']]
X = X.dropna(how = 'any')
X=pd.get_dummies(X)
## Split data into train and test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X.iloc[:, 1:], X[['ArrDelayMinutes']], 
                                                    shuffle=True, random_state=627, test_size=0.2)

In [24]:
## Create an instance
from sklearn.ensemble import RandomForestRegressor
forest_regr = RandomForestRegressor()

In [25]:
scoreForest = np.zeros((5))

In [26]:
##k-fold cross validation
from sklearn.model_selection import KFold
kfold = KFold(n_splits=5,shuffle=True, random_state=626)

for i, (train_index, test_index) in enumerate(kfold.split(X_train, y_train)):
    ## get the kfold training data
    X_train_train = X_train.iloc[train_index, :]
    y_train_train = y_train.iloc[train_index]

    ## get the holdout data
    X_holdout = X_train.iloc[test_index, :]
    y_holdout = y_train.iloc[test_index]

    ## Fit using Dummy Regressor
    forest_regr.fit(X_train_train, y_train_train)
    
    ## Generate predictions on the holdout set
    forest_pred = forest_regr.predict(X_holdout)    

    ## Record mean accuracy of self.predict(X) w.r.t. y as a float
    scoreForest[i] = forest_regr.score(X_holdout, y_holdout) 

C:\Users\linathan\AppData\Local\anaconda3\Lib\site-packages\sklearn\base.py:1365: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


KeyboardInterrupt: 

In [ ]:
scoreForest

In [28]:
X = li[0][['Reporting_Airline','Origin','Dest', 'ArrDelayMinutes']]
X = X.dropna(how = 'any')
X=pd.get_dummies(X)
## Split data into train and test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X.iloc[:, 1:], X[['ArrDelayMinutes']], 
                                                    shuffle=True, random_state=627, test_size=0.2)

from sklearn.linear_model import LinearRegression
linear_regr = LinearRegression()

scoreLinear = np.zeros((5))

##k-fold cross validation
from sklearn.model_selection import KFold
kfold = KFold(n_splits=5,shuffle=True, random_state=626)

for i, (train_index, test_index) in enumerate(kfold.split(X_train, y_train)):
    ## get the kfold training data
    X_train_train = X_train.iloc[train_index, :]
    y_train_train = y_train.iloc[train_index]

    ## get the holdout data
    X_holdout = X_train.iloc[test_index, :]
    y_holdout = y_train.iloc[test_index]

    ## Fit using Dummy Regressor
    linear_regr.fit(X_train_train, y_train_train)
    
    ## Generate predictions on the holdout set
    linear_pred = linear_regr.predict(X_holdout)    

    ## Record mean accuracy of self.predict(X) w.r.t. y as a float
    scoreLinear[i] = linear_regr.score(X_holdout, y_holdout) 

scoreLinear

array([0.01632486, 0.01692929, 0.0184221 , 0.01739314, 0.01704852])